## Define classes to make the indexing less messy

In [119]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: 
            return Sentiment.POSITIVE

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews

    def get_text(self):
        return [x.text for x in self.reviews]

    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    # make negative and positive reviews even so model can be more accurate
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

## Create a list of reviews from the json file

In [105]:
import json
file_name = 'Books_Small_10000.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

print(reviews[5].text)
print(reviews[5].score)
print(reviews[5].sentiment)

I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia's trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character's voice on a strong subject and making it so that other peoples story may be heard through Mia's.
5.0
POSITIVE


## Split data using train_test_split

In [124]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

## Train bag of books model on training data

In [128]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


## Vecotrize text with CountVectorizer

In [129]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)   # do not fit as this is the testing data, we fit on our training data

print(train_x[0])
print(train_x_vectors[0].toarray())

Very enjoyable - recommend if you need something light yet intriguing.Good to take on a vacation. Good for people who work for &#34;the devil&#34;!
[[0 0 0 ... 0 0 0]]


## Classification

### Linear SVM

In [130]:
from sklearn import svm

clf_svm =svm.SVC(kernel='linear')

clf_svm.fit(train_x_vectors, train_y)

test_x[0]

clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Decision Tree

In [131]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Naive Bayes

In [132]:
from sklearn.naive_bayes import GaussianNB

clf_nb = GaussianNB()
clf_nb.fit(train_x_vectors.toarray(), train_y)

clf_nb.predict(test_x_vectors[0].toarray())

array(['POSITIVE'], dtype='<U8')

### Logistic Regression

In [133]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression()
clf_lr.fit(train_x_vectors, train_y)

clf_lr.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

### KNN

In [134]:
from sklearn.neighbors import KNeighborsClassifier

clf_neigh = KNeighborsClassifier()
clf_neigh.fit(train_x_vectors, train_y)

clf_neigh.predict(test_x_vectors[0])

array(['NEGATIVE'], dtype='<U8')

## Evalutaion

In [135]:
# mean accuracy
print(clf_svm.score(test_x_vectors, test_y))
print(clf_neigh.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_nb.score(test_x_vectors.toarray(), test_y))
print(clf_lr.score(test_x_vectors, test_y))

0.7124242424242424
0.5342424242424243
0.5887878787878787
0.44575757575757574
0.7448484848484849


In [136]:
# F1 score
from sklearn.metrics import f1_score

# svm only good for predicting positive
print(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))

print(f1_score(test_y, clf_neigh.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))

print(f1_score(test_y, clf_lr.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))

print(f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE]))

[0.85363477 0.         0.28146853]
[0.70960699 0.         0.16283186]
[0.8783008  0.         0.31077216]
[0.75732218 0.         0.17792642]


## Likely a Data Issue

In [89]:
train_y.count(Sentiment.POSITIVE)   # model heavily biased because there are so many posotives

552